In [1]:
#Membrane simulation of a cyclic peptide generated from Ohue Lab's cyclic peptide binder hallucination protocol.
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.1.0-1/Mambaforge-23.1.0-1-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:17
🔁 Restarting kernel...


In [1]:
!conda install -c "omnia/label/dev" openmmforcefields

Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ failed with initial frozen solve. Retrying with flexible solve.
Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - 

In [3]:
!conda install -c conda-forge rdkit

Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 23.7.3

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
cd drive/MyDrive

/content/drive/MyDrive


In [6]:
!pip install openmm

In [71]:
import openmm.app
from openmm.app import ForceField, Modeller, PME, Simulation, PDBFile, PDBReporter, StateDataReporter
from openmm import unit
import mdtraj as md
import openmm as mm
import numpy as np
import sys

In [8]:
!conda install -c conda-forge pdbfixer

Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 23.7.3

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda up

In [9]:
import pdbfixer
def prepare_protein(
    pdb_path, ignore_missing_residues=False, ignore_terminal_missing_residues=False, ph=7.0):
    """
    Use pdbfixer to prepare the protein from a PDB file. Hetero atoms such as ligands are
    removed and non-standard residues replaced. Missing atoms to existing residues are added.
    Missing residues are ignored by default, but can be included.
    Parameters
    ----------
    pdb_file: pathlib.Path or str
        PDB file containing the system to simulate.
    ignore_missing_residues: bool, optional
        If missing residues should be ignored or built.
    ignore_terminal_missing_residues: bool, optional
        If missing residues at the beginning and the end of a chain should be ignored or built.
    ph: float, optional
        pH value used to determine protonation state of residues
    Returns
    -------
    fixer: pdbfixer.pdbfixer.PDBFixer
        Prepared protein system.
    """

    pdb_file=str(pdb_path)
    fixer = pdbfixer.PDBFixer(str(pdb_file))
    fixer.removeHeterogens(keepWater=False)  # co-crystallized ligands are unknown to PDBFixer, and removing water
    fixer.findMissingResidues()  # identify missing residues, needed for identification of missing atoms

    # if missing terminal residues shall be ignored, remove them from the dictionary
    if ignore_terminal_missing_residues:
        chains = list(fixer.topology.chains())
        keys = fixer.missingResidues.keys()
        for key in list(keys):
            chain = chains[key[0]]
            if key[1] == 0 or key[1] == len(list(chain.residues())):
                del fixer.missingResidues[key]

    # if all missing residues shall be ignored ignored, clear the dictionary
    if ignore_missing_residues:
        fixer.missingResidues = {}


    fixer.findNonstandardResidues()  # find non-standard residue
    fixer.replaceNonstandardResidues()  # replace non-standard residues with standard one
    fixer.findMissingAtoms()  # find missing heavy atoms
    fixer.addMissingAtoms()  # add missing atoms and residues
    fixer.addMissingHydrogens(ph)  # add missing hydrogens
    return fixer


#prepare protein and build only missing non-terminal residues
prepared_protein = prepare_protein("binder.pdb", ignore_missing_residues=False, ph=7.0)

PDBFile.writeFile(prepared_protein.topology, prepared_protein.positions, open("binder_prepared.pdb", 'w'),keepIds=True)


In [78]:
# Load the cyclic peptide from a PDB file
pdb_file_path = 'binder_prepared.pdb'
pdb = PDBFile(pdb_file_path)

In [81]:
from openmm import unit

# Load the molecule from the PDB file using OpenMM's Topology
pdb_topology = pdb.topology

# Get positions of the protein atoms
protein_positions = pdb.positions

# Define the shift vector for the protein
shift_vector = [0.5, 0.25, 0.05]*unit.nanometers

# Apply the shift to protein atom positions
for i in protein_positions:
      i=i+shift_vector

# Create a new Modeller with modified positions
shifted_modeller = Modeller(pdb.topology, protein_positions)

# Load the force field and generate SMIRNOFF parameters
ff = ForceField('amber14-all.xml', 'amber14/tip3p.xml')

# Add the membrane to the modeller

shifted_modeller.addMembrane(ff,lipidType='POPC',membraneCenterZ=0.5*unit.nanometers)

# Create the OpenMM system
system = ff.createSystem(shifted_modeller.topology, nonbondedMethod=PME)

# Set up the integrator and simulation
integrator = mm.LangevinIntegrator(
    300 * unit.kelvin, 1.0 / unit.picosecond, 2.0 * unit.femtoseconds
)
simulation = Simulation(shifted_modeller.topology, system, integrator)
simulation.context.setPositions(shifted_modeller.positions)

# Minimize energy
simulation.minimizeEnergy()

# Set up reporters
steps = 100
write_interval = 1
log_interval = 1

simulation.reporters.append(
    StateDataReporter(
        sys.stdout,
        log_interval,
        step=True,
        potentialEnergy=True,
        temperature=True,
        progress=True,
        remainingTime=True,
        speed=True,
        totalSteps=steps,
        separator="\t",
    )
)
simulation.reporters.append(PDBReporter('binder_md.pdb', 2))

simulation.context.setVelocitiesToTemperature(300 * unit.kelvin)
simulation.step(steps)

#"Progress (%)"	"Step"	"Potential Energy (kJ/mole)"	"Temperature (K)"	"Speed (ns/day)"	"Time Remaining"
1.0%	1	-369915.3817167629	250.96582221236025	0	--
2.0%	2	-359176.56422432204	213.73000120885857	0.162	1:44
3.0%	3	-351023.7505150729	184.11068555723492	0.235	1:11
4.0%	4	-333629.3464240209	141.16303800439462	0.203	1:21
5.0%	5	-348733.9639575966	175.9764445295432	0.232	1:10
6.0%	6	-345864.1885254056	171.39300849171184	0.193	1:24
7.0%	7	-338406.97367731325	153.8761399102501	0.212	1:15
8.0%	8	-347442.45346404513	172.1022239547806	0.201	1:19
9.0%	9	-334794.56020171486	143.90413892271002	0.219	1:11
10.0%	10	-341014.08882287005	159.5928309024263	0.227	1:08
11.0%	11	-349111.2274892642	180.17820905374595	0.243	1:03
12.0%	12	-337223.2839157389	154.3156588954159	0.248	1:01
13.0%	13	-345516.16552141757	171.3392959096256	0.263	0:57
14.0%	14	-339466.0369538258	156.6705785001652	0.258	0:57
15.0%	15	-333388.349785842	143.20825577254135	0.271	0:54
16.0%	16	-346163.6450630155	173.22659066541968	0.273

In [82]:
from google.colab import files
files.download('binder_md.pdb')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>